# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load Preprocessed data

In [2]:
try:
    data = pd.read_csv('../data/adj_close.csv', index_col=0, parse_dates=True)
    # Ensure column names are simplified for easier access if they are still tuples from previous runs
    if isinstance(data.columns, pd.MultiIndex):
        new_cols = []
        for col_tuple in data.columns:
            if 'TSLA' in col_tuple:
                new_cols.append('TSLA')
            elif 'BND' in col_tuple:
                new_cols.append('BND')
            elif 'SPY' in col_tuple:
                new_cols.append('SPY')
            else:
                new_cols.append('_'.join(map(str, col_tuple)).replace('UNNAMED: ', '').replace('_LEVEL_2', ''))
        data.columns = new_cols
        data = data[['TSLA', 'BND', 'SPY']]
    print("Data loaded successfully from adj_close.csv!")
    print("Columns in loaded data:", data.columns)
except Exception as e:
    print(f"Error loading data from adj_close.csv: {e}")
    try:
        raw_data = pd.read_csv('financial_data.csv', header=[0, 1], index_col=0, parse_dates=True)
        data = raw_data['Adj Close']
        data.columns = ['BND', 'SPY', 'TSLA']
        print("Successfully re-extracted 'Adj Close' from 'financial_data.csv'.")
    except Exception as e_fallback:
        print(f"Fallback failed: {e_fallback}. Cannot proceed without proper data.")
        exit()

Data loaded successfully from adj_close.csv!
Columns in loaded data: Index(['('CLOSE', 'BND', 'UNNAMED: 1_LEVEL_2')',
       '('CLOSE', 'SPY', 'UNNAMED: 2_LEVEL_2')',
       '('CLOSE', 'TSLA', 'UNNAMED: 3_LEVEL_2')'],
      dtype='object')


# Backtesting

In [10]:

# --- 1. Define a Backtesting Period ---
backtest_start_date = '2024-08-01'
backtest_end_date = '2025-07-31'

# Filter data for the backtesting period
backtest_data = data[backtest_start_date:backtest_end_date]
print(f"\nBacktesting data from {backtest_start_date} to {backtest_end_date}.")
print(f"Shape of backtesting data: {backtest_data.shape}")

# Calculate daily returns for the backtesting period
backtest_daily_returns = backtest_data.pct_change().dropna()

# Rename columns to simple names for easier selection
backtest_daily_returns.columns = ['BND', 'SPY', 'TSLA']

# --- 2. Define a Benchmark Portfolio ---
# Benchmark: Static 60% SPY / 40% BND
benchmark_weights = np.array([0.0, 0.40, 0.60]) # [TSLA, BND, SPY]
benchmark_assets = ['BND', 'SPY'] # Assets used in benchmark

# Select columns using simple names
benchmark_returns = backtest_daily_returns[benchmark_assets]

# Calculate benchmark portfolio daily returns
benchmark_portfolio_daily_returns = (
    benchmark_returns['BND'] * benchmark_weights[1] +
    benchmark_returns['SPY'] * benchmark_weights[2]
)

# Calculate benchmark cumulative returns
benchmark_cumulative_returns = (1 + benchmark_portfolio_daily_returns).cumprod() - 1
print("\nBenchmark Portfolio Defined (60% SPY / 40% BND).")

# --- 3. Simulate Your Strategy Portfolio ---
# Use the optimal weights from the Maximum Sharpe Ratio Portfolio found in Task 4.
# From the previous output:
# TSLA: 0.0000
# BND: 0.0429
# SPY: 0.9571
strategy_weights = np.array([0.0000, 0.0429, 0.9571]) # [TSLA, BND, SPY]
strategy_assets = ['TSLA', 'BND', 'SPY']
strategy_returns = backtest_daily_returns[strategy_assets]

# Calculate strategy portfolio daily returns
strategy_portfolio_daily_returns = (
    strategy_returns['TSLA'] * strategy_weights[0] +
    strategy_returns['BND'] * strategy_weights[1] +
    strategy_returns['SPY'] * strategy_weights[2]
)

# Calculate strategy cumulative returns
strategy_cumulative_returns = (1 + strategy_portfolio_daily_returns).cumprod() - 1
print("Strategy Portfolio Defined (Max Sharpe Ratio from Task 4).")



Backtesting data from 2024-08-01 to 2025-07-31.
Shape of backtesting data: (249, 3)

Benchmark Portfolio Defined (60% SPY / 40% BND).
Strategy Portfolio Defined (Max Sharpe Ratio from Task 4).
